In [0]:
# !apt-get install libxml2-dev libxslt-dev
# !apt-get install libjpeg-dev zlib1g-dev libpng12-dev
# !pip install --upgrade setuptools
# !pip3 install newspaper3k
# !pip3 install cassandra-driver
# !curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3
# !pip install xlrd

In [0]:
import requests
import json
import random
from newspaper import Article
import xlrd

import pandas as pd
import numpy as np
import random

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer

from keras import optimizers
from keras.models import load_model
from keras.models import Model, Input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import multi_gpu_model, to_categorical
from keras.layers import Dense, Embedding, LSTM, GRU, Dropout, Bidirectional

/Users/annie/anaconda/envs/update_AdvML/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
# from google.colab import files
# uploaded = files.upload()

In [0]:
excelLocation = "Book1.xlsx" 
wb = xlrd.open_workbook(excelLocation)
sheet = wb.sheet_by_index(0)
 
print(sheet.ncols)

sheet.cell_value(0, 0)

4


'symbol'

In [0]:
#Qian's function goes here
def getSentiment(model_file, input_texts, max_vocab = 2000, max_len = 200):
    
    # Reload model
    model = load_model(model_file)
    #print(model.summary())
#     input_texts = pd.Series(input_texts)
#     input_texts = input_texts.apply(lambda x: x.lower())
#     input_texts = input_texts.apply(lambda x: x.replace("\n", ""))
#     input_texts = input_texts.apply(lambda x: x.replace("\'", ""))
#     print(input_texts)
    
    # Preprocess input texts
    tokenizer = Tokenizer(num_words = max_vocab, split = ' ', char_level = False)
    tokenizer.fit_on_texts(input_texts)
    X = tokenizer.texts_to_sequences(input_texts)
    X = pad_sequences(X, maxlen = max_len, padding = 'post', truncating = 'post')

    # Predict label
    output_label = model.predict(X)
    output_label = np.argmax(output_label, axis = 1)
    
#     print("Predited labels: ", output_label)
    
    return(output_label)

In [0]:
def getArticleContent(url):
    article = None
    article = Article(url)
    article.download()
    article.parse()
    return article.text

In [0]:
# data = {}

# for i in range(1, sheet.nrows):
#         coinName = sheet.cell_value(i, 0)
#         searchKey = sheet.cell_value(i, 2)
#         data[searchKey] = []
#         APIKey = sheet.cell_value(i, 1)

#         baseURL = 'https://newsapi.org/v2/everything?sortBy=publishedAt&apiKey=' + APIKey + '&q=' + searchKey 

#         if sheet.ncols > 2:
#             for k in range(3, sheet.ncols):
#                 parameter = sheet.cell_value(0, k)
#                 value = sheet.cell_value(i, k)
#                 appendToURL = '&' + parameter + '=' + value
#                 baseURL += appendToURL

#             #   print (baseURL)

#         response = requests.get(baseURL)
#         jsonVal = response.json()

#      #   print (jsonVal)

#         articlesTemp = jsonVal['articles']
# #        for i in articlesTemp:
#         for j in range(len(articlesTemp)):
# #             try:
# #                 url = i['url']
# #                 publishedAt = i['publishedAt']
# #                 title = i['title']
# #                 description = i['description']
# #                 author = i['author']
# #                 content = getArticleContent(url).strip()
#                 url = articlesTemp[j]['url']
#                 publishedAt = articlesTemp[i]['publishedAt']
#                 title = articlesTemp[j]['title']
#                 description = articlesTemp[i]['description']
#                 author = articlesTemp[j]['author']
#                 content = getArticleContent(url).strip()
#                 content = pd.Series(content)
                
#                 content = content.apply(lambda x: x.lower())
#                 content = content.apply(lambda x: x.replace("\n", ""))
#                 content = content.apply(lambda x: x.replace("\'", ""))
                

#                 tempDict = {}
#                 tempDict['url'] = url
#                 tempDict['publishedAt'] = publishedAt
#                 tempDict['title'] = title
#                 tempDict['description'] = description
#                 tempDict['author'] = author
#                 tempDict['content'] = content
#                 tempDict['coin'] = coinName
#                 print(tempDict['url'])
#                 print(tempDict['content'])
#                 tempDict['sentiment'] = getSentiment(model_file = 'blstm3_w_dp_gpu_200_100_1024_fold_3.h5', 
#                                                      input_texts = tempDict['content'], 
#                                                      max_vocab = 2000, 
#                                                      max_len = 200)
#                 print(tempDict['sentiment'])
                
    

#                 if content != "":
#                     data[searchKey].append(tempDict)

# #             except:
# #                 continue

In [0]:
data = {}

for i in range(1, sheet.nrows):
        coinName = sheet.cell_value(i, 0)
        searchKey = sheet.cell_value(i, 2)
        data[searchKey] = []
        APIKey = sheet.cell_value(i, 1)

        baseURL = 'https://newsapi.org/v2/everything?sortBy=publishedAt&apiKey=' + APIKey + '&q=' + searchKey 

        if sheet.ncols > 2:
            for k in range(3, sheet.ncols):
                parameter = sheet.cell_value(0, k)
                value = sheet.cell_value(i, k)
                appendToURL = '&' + parameter + '=' + value
                baseURL += appendToURL

            #   print (baseURL)

        response = requests.get(baseURL)
        jsonVal = response.json()


        articlesTemp = jsonVal['articles']
        tempFrame = pd.DataFrame(columns = ['url', 'publishedAt', 'title', 'description', 'author', 'content', 'coin'])

        for j in range(len(articlesTemp)):
            
                url = articlesTemp[j]['url']
                publishedAt = articlesTemp[i]['publishedAt']
                title = articlesTemp[j]['title']
                description = articlesTemp[i]['description']
                author = articlesTemp[j]['author']
                content = getArticleContent(url).strip()
                
                content = pd.Series(content)
                content = content.apply(lambda x: x.lower())
                content = content.apply(lambda x: x.replace("\n", ""))
                content = content.apply(lambda x: x.replace("\'", ""))
                tempFrame.loc[j] = [url, publishedAt, title, description, author, content[0], coinName]
            
                
        tempFrame['sentiment'] = getSentiment(model_file = 'blstm3_w_dp_gpu_200_100_1024_fold_3.h5', 
                                              input_texts = tempFrame['content'], 
                                              max_vocab = 2000, 
                                              max_len = 200)
        print(tempFrame.to_dict('list'))
        data[searchKey].append(tempFrame.to_dict('list'))
        

        
    

Predited labels:  [2 1 0 0 0 1 2 2 1 1 0 1 2 0 0 1 1 1 2 2]
{'url': ['https://www.newsbtc.com/2018/08/25/reddit-co-founder-bullish-on-bitcoin-despite-current-price-slump/', 'https://indianexpress.com/article/cities/fake-call-centre-busted-in-thane-seven-held-5324930/', 'https://www.ccn.com/why-some-bulls-expect-bitcoin-to-fall-below-5000-prior-to-big-rally/', 'https://www.ccn.com/etf-rejections-printing-money-and-the-woz-this-week-in-crypto/', 'https://bitcoinist.com/bitcoin-price-defies-sec-etf-rejection/', 'http://fortune.com/2018/08/25/the-ledger-cryptocurrency-tesla-bitcoin-etf-indiegogo/', 'https://www.forbes.com/sites/jasonbloomberg/2018/08/25/dr-doom-economist-nouriel-roubini-bearish-on-everything-crypto/', 'https://www.ccn.com/u-s-senator-warns-closing-montana-coal-plant-will-hurt-bitcoin-mining-industry/', 'https://www.ccn.com/sec-will-review-rejected-bitcoin-etfs-why-they-will-remain-rejected/', 'https://hazlitt.net/feature/state-bestiary-stable', 'https://patentlyo.com/paten

Predited labels:  [2 0 1 2 2 2 0 1 1 2 1 1 0 2 2 1 1 2 2 1]
{'url': ['https://www.ccn.com/why-some-bulls-expect-bitcoin-to-fall-below-5000-prior-to-big-rally/', 'https://bitcoinist.com/bitcoin-price-defies-sec-etf-rejection/', 'https://southfront.org/chinese-naval-expansion-hits-high-gear/', 'https://cointelegraph.com/news/markets-see-wave-of-green-btc-pushing-6-750-eth-at-280', 'https://bitcoinist.com/bitcoin-60k-when-not-if-bobby-lee/', 'https://www.autosport.com/btcc/news/138266/cammish-beats-bmws-for-maiden-btcc-pole', 'https://www.ccn.com/215-billion-crypto-market-continues-recovery-as-bitcoin-price-hits-6700/', 'https://bitcoinmagazine.com/articles/bitcoin-magazines-week-review-rejections-and-reflections/', 'https://www.newsbtc.com/2018/08/25/cnbc-tweets-have-been-a-contrarian-bitcoin-price-indicator-with-95-accuracy/', 'https://www.youtube.com/watch?v=brlQ2B9s2Wc', 'https://www.newsbtc.com/2018/08/25/crypto-markets-sluggish-on-saturday-as-bitcoin-increases-dominance/', 'https://

Predited labels:  [0 2 2 2 1 1 2 0 2 2 2 1 2 2 2 1 1 0 0 1]
{'url': ['https://www.ccn.com/etf-rejections-printing-money-and-the-woz-this-week-in-crypto/', 'https://www.forbes.com/sites/jasonbloomberg/2018/08/25/dr-doom-economist-nouriel-roubini-bearish-on-everything-crypto/', 'https://cointelegraph.com/news/markets-see-wave-of-green-btc-pushing-6-750-eth-at-280', 'https://www.ccn.com/215-billion-crypto-market-continues-recovery-as-bitcoin-price-hits-6700/', 'https://www.ccn.com/hackers-hijack-elon-musks-twitter-offer-free-cryptocurrency/', 'https://www.breitbart.com/tech/2018/08/25/ripple-xrp-more-decentralized-bitcoin/', 'https://www.coindesk.com/the-weirdest-prediction-markets-on-augur-right-now/', 'https://hackernoon.com/turkeys-economic-crisis-can-trigger-the-next-crypto-bull-run-86e53392d9bc', 'https://www.newsbtc.com/2018/08/25/crypto-markets-sluggish-on-saturday-as-bitcoin-increases-dominance/', 'https://medium.com/@umcompany.mail/the-situation-of-the-cryptocurrency-market-in-th

In [0]:
#Remove this code when going to production. 
#This just prints out all articles content
for i in data.keys():
    for j in data[i]:
        print (j['content'])

['alexis ohanian, the co-founder of popular news and forum site, reddit, is bullish on bitcoin despite the current price slump. his optimistic views on the future of bitcoin come as he moves away from reddit and towards more venture capital initiatives, including investments in cryptocurrencies and cryptocurrency companies.following the acquisition of reddit by conde nast parent company, advance publications, ohanian has been slowly relieving himself of his day-to-day duties at the online platform. the 35-year old is now looking towards investing in more cutting-edge technologies, including instacart, and self-driving car company cruise. he has also been moving further into the cryptocurrency industry, acquiring a sizable amount of bitcoin as well as an investment in cryptocurrency exchange coinbase.ohanian is optimistic about his investments on both a fiscal and personal level, explaining that his new job as the head of his venture capital firm, initialized capital, allows for better 

Cassandra Python connector
http://datastax.github.io/python-driver/getting_started.html

In [0]:
from cassandra.cluster import Cluster
cluster = Cluster()   
#Cluster([IP1, IP2,...])

session = cluster.connect()
#cluster.connect(KEYSPACENAME)


#Sample insert query for cassandra
#Discuss the structure with team once
session.execute(
    """
    INSERT INTO users (name, credits, user_id)
    VALUES (%s, %s, %s)
    """,
    ("John O'Reilly", 42, uuid.uuid1())
)